In [1]:
# Import general libraries
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

In [2]:
%%time
# Import velocity field
import scipy.io as sio

print("Import data")
#Import velocity data from file in data-folder
mat_file = sio.loadmat('./data/Files/Bickley_jet.mat')

U = mat_file['u']
V = mat_file['v']
x = mat_file['x']
y = mat_file['y']
time = mat_file['t']

Import data
Wall time: 41.9 ms


In [3]:
# Create meshgrid
X, Y = np.meshgrid(x, y)

In [4]:
# Name of Dataset. {"Ocean", "General"}
Type = "General"

# Number of cores to be used for parallel computing
Ncores = 4

# Specify whether the flow is incompressible. {True, False}
incompressible = True

# Specify periodic boundary conditions
periodic_boundary = [False, False]

# Summarize list of parameters of the flow.
params_data = Type, X, Y, time, U, V, Ncores, incompressible, periodic_boundary

In [5]:
%%time
import sys
sys.path.append("./classes")
from ipynb.fs.defs.Dynamical_System import *
DS = Dynamical_System()
DS._init_data(params_data)

Wall time: 76.8 ms


In [6]:
DS._Interpolation_velocity()

In [7]:
%%time

# Initial time (in days)
t0 = DS.Time[0, 0]

# Final time (in days)
tN = DS.Time[0, 15]

# time step-size
dt = 0.2

time = np.arange(t0, tN+dt, dt)

#  x, y boundaries
xmin = np.min(DS.X)
xmax = np.max(DS.X)/20
ymin = np.min(DS.Y)/2
ymax = np.max(DS.Y)/2

# spacing of meshgrid (in degrees)
dx = np.minimum(X[0, 1]-X[0, 0], Y[1, 0]-Y[0, 0])/10
dy = dx

# x, y grid with spacing dx, dy
x_grid = np.arange(xmin, xmax+dx, dx)
y_grid = np.arange(ymin, ymax+dy, dy)

# Specify ratio of auxiliary grid
ratio_auxiliary_grid = 0.1

params_ds = time, x_grid, y_grid, ratio_auxiliary_grid
DS._init_ds(params_ds)

Wall time: 995 µs


In [ ]:
FTLE_min, FTLE_max = DS._FTLE_()

=================Trajectories over meshgrid=================


  0%|          | 0/96 [00:00<?, ?it/s]

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
%matplotlib notebook 
%matplotlib notebook 

from IPython.display import HTML
    
c = DS.FTLE[:,:,1].ravel()
x = DS.trajectory_grid[:,:,0,0].ravel()
y = DS.trajectory_grid[:,:,1,0].ravel()

steps = np.arange(0, DS.lenT, 1)
    
# create figure and axes 
fig, ax = plt.subplots(1, figsize=(8, 4))
scat = ax.scatter(x, y, c=c, s=6)
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
    
def update_plot(i, X, Y, scat):
    x_data = X[:,:,i].ravel()
    y_data = Y[:,:,i].ravel()
    ax.set_title(f'$FTLE_{{{DS.t0}}}^{{{DS.tN}}}(t = {{{np.around(DS.time[i], 2)}d}})$')
    ax.set_xlim(np.nanmin(x_data), np.nanmax(x_data))
    ax.set_ylim(np.nanmin(y_data), np.nanmax(y_data))
    scat.set_offsets(np.column_stack([x_data, y_data]))
    return scat,
    
# the FuncAnimation function iterates through our animate function using the steps array
ani = animation.FuncAnimation(fig, update_plot, frames=steps,
                                  fargs=(DS.trajectory_grid[:,:,0,:], DS.trajectory_grid[:,:,1,:], scat), 
                                  interval = 20, 
                                  blit = True)
    
HTML(ani.to_jshtml())
# the FuncAnimation function iterates through our animate function using the steps array
#ani = FuncAnimation(fig, animate, steps, init_func=init, blit=True)

In [ ]:
PRA = DS._PRA_()

In [ ]:
fig = plt.figure(figsize = (8, 4))
ax = plt.axes()
cax = ax.contourf(DS.X_domain, DS.Y_domain, PRA, levels = 600, cmap = "gist_rainbow")
#cax = ax.scatter(DS.trajectory_grid[:, :, 0, 0].ravel(), DS.trajectory_grid[:, :, 1, 0].ravel(), c = FTLE_max.ravel())
plt.colorbar(cax, ticks = np.linspace(0, np.pi, 8))
plt.show()

In [ ]:
#extrema_x, extrema_y = DS._ridge_trench(FTLE_max, threshold = 0.02, type = "ridge", method = "gradient", 
#                                        resolution = 1, ds = 1, n_iterations = 100)
#plt.scatter(extrema_x, extrema_y, s = .5)
#plt.show()